In [92]:
import os
import sys
import pandas as pd
import numpy as np
import pickle

In [93]:
import findspark
findspark.init()
from pyspark import *
sc = SparkContext.getOrCreate()

In [94]:
usage_rdd = sc.textFile('../resources/data0117/').map(lambda x: x.split('|'))
usage_rdd = usage_rdd.filter(lambda x: x[2] != '柚宝宝孕育')
usage_rdd.take(2)

[['C966BB7B4BB196602D1054B27E6F9BAE', '10', 'QQ', '通讯聊天', '15'],
 ['C2827EBF2B61C4FB71C164F3D13BE312', '08', '微信', '通讯聊天', '1']]

In [95]:
data0117 = pd.DataFrame(usage_rdd.collect(), columns=['id','hour','app','app_cat','times'])
data0117.head()

,id,hour,app,app_cat,times
0,C966BB7B4BB196602D1054B27E6F9BAE,10,QQ,通讯聊天,15
1,C2827EBF2B61C4FB71C164F3D13BE312,08,微信,通讯聊天,1
2,F819FE792867382C9C8BD39BA7837746,21,QQ,通讯聊天,4
3,DA549AD85F2DD7E87126416CCA4C44B4,18,微信,通讯聊天,3
4,F81A9E1BCC2FF6820539952284E9C7A9,12,开心消消乐,游戏服务,1


In [122]:
# 确保数据类型
data0117['hour'] = data0117['hour'].astype(int)
data0117['times'] = data0117['times'].astype(int)

In [135]:
len(data0117)
print(data0117.dtypes)

id         object
hour        int64
app        object
app_cat    object
times       int64
dtype: object


### 1. 用户的App安装列表

In [97]:
installed_apps = data0117.groupby('id', as_index=False)['app'].agg({'installed_apps': lambda x: set(x.tolist())})
installed_apps.head()

,id,installed_apps
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,"{微信, 百度云, 支付宝, Keep, 开心消消乐, 金山WPS OFFICE手机版, 百..."
1,B02B18790004038B27DDBEB461D2EAD8,"{微信, 大众点评网, 腾讯视频, QQ, 贪吃蛇大作战, 百度搜索}"
2,B02B7DD46B57A2374C936C799377A183,"{钉钉, 百度搜索, 爱奇艺, 微信}"
3,B02BA3E7E93A7BC84B5669C974FB0FAE,"{微信, 携程旅行, 京东商城, 支付宝, 网易云音乐, Keep, 腾讯视频, QQ, 大..."
4,B02BCC791EB79100420BF041BAF31D4F,"{微信, 京东商城, 支付宝, Keep, 大众点评网, QQ, 腾讯视频, 开心消消乐, ..."


In [98]:
len(installed_apps)

25413

---
2. 用户当前时刻使用的App列表
---
这一特征需要对数据进行按用户和时间进行排序，然后取时间偏置merge到一起

---

In [136]:
positive_apps = data0117.groupby(['id', 'hour'], as_index=False)['app'].agg({'positive_apps': lambda x: set(x.tolist())})
positive_apps.head()

,id,hour,positive_apps
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,{百度搜索}
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,"{微信, 支付宝, Keep, 开心消消乐, 百度搜索, 搜狗输入法, 今日头条}"
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,8,"{微信, 支付宝, Keep, 百度搜索, 搜狗输入法, 今日头条}"
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,9,"{微信, 搜狗输入法, 今日头条}"
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,10,"{微信, 搜狗输入法, 今日头条, 支付宝}"


In [100]:
temp = positive_apps['positive_apps'].apply(lambda x: len(x))
min(temp), max(temp)

(1, 19)

In [149]:
temp['last_apps'].describe()

count    297139.000000
mean          3.196847
std           2.149925
min           1.000000
25%           1.000000
50%           3.000000
75%           4.000000
max          19.000000
Name: last_apps, dtype: float64

---
3. App属性特征
---
名称|类别|开发者|评分|下载量|描述

---

In [101]:
app_cat = data0117.groupby('app', as_index=False).agg({'app_cat': lambda x: ','.join(set(x.tolist()))})
print(len(app_cat))

54


In [102]:
applist = list(app_cat['app'])
def edit_distance(s1, s2):
    len1 = len(s1)
    len2 = len(s2)
    dp = []
    for i in range(len1+1):
        dp.append([])
        for j in range(len2 + 1):
            dp[i].append(0)
    for i in range(len1 + 1):
        dp[i][0] = i
    for j in range(len2 + 1):
        dp[0][j] = j
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i-1][j-1], dp[i-1][j], dp[i][j-1]) + 1
    return dp[len1][len2]

import numpy as np
def matchApp(name, applist):
    ind = np.argmin(list(map(lambda app: edit_distance(name, app), applist)))
    return applist[ind]

import collections
appAttr = collections.defaultdict(list)
with open('../resources/app_yingyongbao.txt', encoding='gbk') as f:
    for line in f:
        if '柚宝宝' in line:
            continue
        splits = line.split('|', maxsplit=5)
        if len(splits) == 6:
            name = splits[0]
            appAttr['name'].append(splits[0])
            appAttr['cat'].append(splits[1])
            appAttr['developer'].append(splits[2])
            appAttr['ratings'].append(float(splits[3].split('=')[1]))
            appAttr['downloads'].append(float(splits[4].split('=')[1]))
            appAttr['description'].append(splits[5])
appAttrDf = pd.DataFrame(appAttr)
appAttrDf = pd.concat([app_cat, appAttrDf], axis=1)

In [103]:
appAttrDf

,app,app_cat,cat,description,developer,downloads,name,ratings
0,58同城,便捷生活,生活,人人信赖的生活服务平台，海量生活信息免费查询、发布。 \r\n真实、高效、免费，让您生活更...,北京五八到家信息技术有限公司,2.406292e+08,58同城 -租房招聘兼职二手车,4.272016
1,Keep,健康医疗,健康,「Keep」— 你的自由运动场，打开 App，即可体验健身、跑步、骑行、计步功能，随时随地练...,北京卡路里科技有限公司,4.288073e+07,Keep,4.194313
2,QQ,通讯聊天,社交,-----QQ乐在沟通-----\r\n\r\n√服务超过90%的移动互联网用户\r\n√多...,腾讯公司,7.451432e+09,QQ,3.417851
3,QQ空间,社交网络,社交,QQ空间，超过6亿用户使用的社交网络。\r\n致力于帮助用户随时随地地“分享生活，留住感动”...,腾讯公司,1.488560e+09,QQ空间,3.866392
4,一号店,电子商务,购物,----全球超市，轻松到家！买更好的东西，上1号店！----\r\n1.专享福利：剁手价、专...,重庆京东海嘉电子商务有限公司,2.289155e+07,1号店,4.256990
5,中华万年历,个人工具,工具,旧貌换新颜，中华万年历品牌全面升级。日历、运势、日程、记事、提醒，功能全，体积小；速度快，口...,北京微鲤科技有限公司,1.040362e+08,中华万年历,4.816138
6,中国国际航空,旅行预订,旅游,中国国际航空公司 Air China是中国唯一的载旗航空公司，星空联盟成员。航线网络覆盖中国...,大连航空有限责任公司,2.311770e+06,中国国航,4.573248
7,京东商城,电子商务,购物,-----挑好物，上京东-----\r\n1.新人大礼：188元新人礼包免费领 \r\n2....,北京京东叁佰陆拾度电子商务有限公司,5.263987e+08,京东,3.824715
8,今日头条,综合资讯,新闻,今天，就看今日头条.今日头条，超过 7 亿用户选择的新闻资讯 App.单用户每日使用时长超过...,北京字节跳动科技有限公司,1.694433e+09,今日头条,4.600000
9,南方航空,旅行预订,旅游,---南方航空---\n天合联盟成员中国南方航空股份有限公司，是中国运输飞机最多，航线网络最...,中国南方航空股份有限公司,3.062855e+06,南方航空,4.735294


---
3. 负采样构建训练集
---
将用户当前时刻使用过app作为正例，对未使用过的app做负采样

---

In [174]:
950000, 18*25413*55  # 所以要进行负采样

(950000, 25158870)

In [104]:
applist = list(set(data0117['app']))
len(applist)

54

In [105]:
import random
def neg_sampling(pos_samples, num_samples=5):
    num_samples = min(num_samples, len(applist) - len(pos_samples))
    neg_samples = []
    while len(neg_samples) < num_samples:
        sample = random.choice(applist)
        if sample in pos_samples or sample in neg_samples:
            continue
        neg_samples.append(sample)
    return neg_samples

In [106]:
neg_sampling(set(['微信', '搜狗输入法', '今日头条']))

['京东商城', '探探', '中国国际航空', '易车', '网易严选']

In [137]:
negative_apps = positive_apps.copy()
negative_apps['negative_apps'] = negative_apps['positive_apps'].apply(lambda x: set(neg_sampling(x)))
negative_apps.drop('positive_apps', axis=1, inplace=True)
negative_apps.head()

,id,hour,negative_apps
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,"{宝宝树孕育, 百度云, 墨迹天气, 咕咚运动, 追书神器}"
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,"{网易云音乐, 饿了么, QQ, 平安好医生, 中国国际航空}"
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,8,"{咕咚运动, 高德地图, 驴妈妈, 卷皮, 贪吃蛇大作战}"
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,9,"{QQ空间, 蘑菇街, 网易严选, 掌阅书城, 百度搜索}"
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,10,"{去哪儿旅行网, 大智慧, 饿了么, 蘑菇街, 平安好医生}"


# TODO 
当前负样本是根据某一时刻的正样本产生的，如果当前时刻没有使用手机还要不要产生负样本

---
4. dataframe文本内容列转行，一行转多行
---
```python
df = pd.DataFrame({'A':['1','2','3'],'B':['1','2,3','4,5,6'],'C':['3','3','3']})
df = (df.set_index(['A','C'])['B']
       .str.split(',', expand=True)
       .stack()
       .reset_index(level=2, drop=True)
       .reset_index(name='B'))
print(df)
```

---

In [138]:
pos = positive_apps.set_index(['id','hour'])['positive_apps'].apply(lambda x: ','.join(x)).str.split(',', expand=True)\
.stack().reset_index(level=2, drop=True).reset_index(name='app')
pos['click'] = 1
pos.head()

,id,hour,app,click
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,百度搜索,1
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,微信,1
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,支付宝,1
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,Keep,1
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,开心消消乐,1


In [139]:
neg = negative_apps.set_index(['id','hour'])['negative_apps'].apply(lambda x: ','.join(x)).str.split(',', expand=True)\
.stack().reset_index(level=2, drop=True).reset_index(name='app')
neg['click'] = 0
neg.head(7)

,id,hour,app,click
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,宝宝树孕育,0
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,百度云,0
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,墨迹天气,0
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,咕咚运动,0
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,追书神器,0
5,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,网易云音乐,0
6,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,饿了么,0


In [247]:
len(pos), len(neg)

(950284, 1485725)

In [140]:
data = pd.concat([pos, neg], axis=0)
data.head()

,id,hour,app,click
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,百度搜索,1
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,微信,1
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,支付宝,1
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,Keep,1
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,开心消消乐,1


In [111]:
data.tail()

,id,hour,app,click
1485690,FFFF65C708A33E03D2DF17A588444F82,22,驾考宝典,0
1485691,FFFF65C708A33E03D2DF17A588444F82,22,支付宝,0
1485692,FFFF65C708A33E03D2DF17A588444F82,22,大智慧,0
1485693,FFFF65C708A33E03D2DF17A588444F82,22,掌阅书城,0
1485694,FFFF65C708A33E03D2DF17A588444F82,22,今日头条,0


In [255]:
len(data)

2436009

---
5. 添加特征
---
将之前生成用户App安装列表，上一时刻使用的App等特征扩展到数据集中

---

In [112]:
temp = installed_apps.copy()
temp['installed_apps'] = temp['installed_apps'].apply(lambda x: len(x))
temp.head()

,id,installed_apps
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,9
1,B02B18790004038B27DDBEB461D2EAD8,6
2,B02B7DD46B57A2374C936C799377A183,4
3,B02BA3E7E93A7BC84B5669C974FB0FAE,12
4,B02BCC791EB79100420BF041BAF31D4F,10


In [113]:
data = pd.merge(data, temp, on=['id'], how='left')
data.head()

,id,hour,app,click,installed_apps
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,06,百度搜索,1,9
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,07,微信,1,9
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,07,支付宝,1,9
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,07,Keep,1,9
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,07,开心消消乐,1,9


In [141]:
# 上一时刻使用app列表，与原dataframe将来merge到一起
temp = positive_apps.copy()
temp['hour'] += 1  # 记录上一时刻使用的App列表
temp.rename(columns={'positive_apps': 'last_apps'}, inplace=True)
temp['last_apps'] = temp['last_apps'].apply(lambda x: len(x))
temp.head()

,id,hour,last_apps
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,1
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,8,7
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,9,6
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,10,3
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,11,4


In [142]:
data = pd.merge(data, temp, on=['id', 'hour'], how='left')
data.head()

,id,hour,app,click,last_apps
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,百度搜索,1,NaN
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,微信,1,1.0
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,支付宝,1,1.0
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,Keep,1,1.0
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,开心消消乐,1,1.0


In [143]:
data.fillna(0, inplace=True)
data.head()

,id,hour,app,click,last_apps
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,百度搜索,1,0.0
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,微信,1,1.0
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,支付宝,1,1.0
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,Keep,1,1.0
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,开心消消乐,1,1.0


In [144]:
data = pd.merge(data, app_cat, on=['app'], how='left')
data.head()

,id,hour,app,click,last_apps,app_cat
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,百度搜索,1,0.0,实用工具
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,微信,1,1.0,通讯聊天
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,支付宝,1,1.0,金融理财
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,Keep,1,1.0,健康医疗
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,开心消消乐,1,1.0,游戏服务


In [290]:
node2id['B02AD69BFD4A4FEEDFAB6DBB36DCC167'], node2id['微信']

(0, 25433)

---
6.添加graph embedding特征
---
该特征是通过对User-App网络进行网络表示学习得到节点的embedding表示，不仅包含低维稠密的信息，更包含节点之间的语义信息

---

In [75]:
node2id = pickle.load(open('../resources/node2id.pkl', 'rb'))
len(node2id)

25486

In [86]:
def hadamard(vec1, vec2):
    return list(map(lambda x: x[0]*x[1], zip(vec1, vec2)))

In [83]:
id2node = {v: k for k, v in node2id.items()}
def get_embeddings(file_name, dummy=True):
    embeddings = {}
    embeddings_file = os.path.join("../network_embedding/embeddings_output/", file_name)
    with open(embeddings_file, 'r') as f:
        for line in f:
            splits = line.split()
            if len(splits) == 2:
                print('node: %s, embedding_size: %s' % tuple(splits))
                continue
            if dummy:
                embeddings[id2node[int(splits[0])]] = list(map(float, splits[1:]))
            else:
                embeddings[splits[0]] = list(map(float, splits[1:]))
    return embeddings

In [84]:
embeddings = get_embeddings('deepwalk.embeddings', dummy=True)

node: 25486, embedding_size: 128


In [90]:
assert '微信' in embeddings, 'error, wechat not in embeddings'

In [145]:
user_apps = data.loc[:, ['id', 'app']].drop_duplicates()
user_apps.head()

,id,app
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,百度搜索
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,微信
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,支付宝
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,Keep
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,开心消消乐


In [146]:
len(temp)

297139

In [147]:
temp = user_apps.apply(lambda row: hadamard(embeddings[row['id']], embeddings[row['app']]), axis=1)

KeyboardInterrupt: 

In [315]:
temp = user_apps['graph_emb'].apply(lambda col: ','.join(map(str, col)))

In [316]:
temp.head()

0    -0.0051594073935908,0.0666825147279899,0.01853...
1    0.00547985938768246,0.08373862471515771,0.0081...
2    -0.0058435140981085995,0.1172621817009695,-0.0...
3    -0.0454774110441126,0.12108299415182061,-0.015...
4    0.00423594427914244,-0.02816071338299894,0.027...
Name: graph_emb, dtype: object

In [320]:
temp = temp.str.split(',', expand=True)
temp.columns = ['emb_%s' % i for i in range(128)]
temp.head()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127
0,-0.0051594073935908,0.0666825147279899,0.018539364552579,0.0114876858774192,0.006336184095452399,-0.005988185346273221,0.008823505582190999,-0.0009744207487502401,0.00275967084127734,0.001258217567854,...,0.053269274503195504,0.0053896737044286,-0.018579164025534,-0.00277981162437032,-0.000163717133677688,0.023285327806744922,-0.00049816619704834,0.0072735408459075005,-2.95532105771628e-05,-0.005862502867530559
1,0.00547985938768246,0.08373862471515771,0.0081825954678621,0.02133262384388,0.010413039525126,4.45402708464813e-05,0.0013803316024664398,-0.00029128177872351596,0.0006449705735577931,-0.004441708427,...,0.05624127880245,0.0050387043105246,0.0174908688997336,-0.000772765778317736,-0.00125468678113906,0.0141222844821219,0.0001524598415045692,0.0066193911949005,-7.976739848230561e-05,-0.00406664138263268
2,-0.0058435140981085995,0.1172621817009695,-0.00040312178643555,0.006498491466652399,0.004801410687309201,0.00304571867157935,0.018736522847407802,-0.0014447045104649821,0.002002165914987009,-0.01653579242614,...,0.06474552556919999,0.0037219867270488,0.020390728582534,-0.004855609726982352,0.0035372942365255204,0.013201818435589,-0.0030404271371136,-0.0032332441818288,-7.10689383652776e-05,-0.0047330230107115595
3,-0.0454774110441126,0.12108299415182061,-0.0150667818984648,0.021571585867524,0.0053595031181847995,-0.0058139890027163,0.0259050511611456,0.00700461267238437,0.0033656174420928302,-0.000864143795372,...,0.0106611927907857,0.0022276670574426,0.018511982163858604,-0.00528872129122992,0.0006997969256685561,-0.01073317343449638,-0.00205497725524466,-0.00242867009111637,0.00020931604237424758,-0.00390062219638372
4,0.00423594427914244,-0.02816071338299894,0.027082874290590003,-0.018418750754026,0.0073998160262664795,-0.012032801379430202,0.00059104008062544,0.001585661214447408,0.002836061504325432,0.007034839892912,...,0.03266895085413,0.0034214489954958,0.057234973404225,0.0007363980335044479,-0.0033633517557612406,-0.015277080881689902,-0.0026509078813809,0.0015645956417787,-0.000214980965065152,-0.003144631266761308


In [324]:
user_apps = pd.concat([user_apps, temp], axis=1)

In [325]:
data = pd.merge(data, user_apps, on=['id', 'app'], how='left')

In [327]:
data.head()

,id,hour,app,click,installed_apps,last_apps,app_cat,emb_0,emb_1,emb_2,...,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127
0,B02AD69BFD4A4FEEDFAB6DBB36DCC167,6,百度搜索,1,9,0.0,实用工具,-0.0051594073935908,0.0666825147279899,0.018539364552579,...,0.053269274503195504,0.0053896737044286,-0.018579164025534,-0.00277981162437032,-0.000163717133677688,0.023285327806744922,-0.00049816619704834,0.0072735408459075005,-2.95532105771628e-05,-0.005862502867530559
1,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,微信,1,9,1.0,通讯聊天,0.00547985938768246,0.08373862471515771,0.0081825954678621,...,0.05624127880245,0.0050387043105246,0.0174908688997336,-0.000772765778317736,-0.00125468678113906,0.0141222844821219,0.0001524598415045692,0.0066193911949005,-7.976739848230561e-05,-0.00406664138263268
2,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,支付宝,1,9,1.0,金融理财,-0.0058435140981085995,0.1172621817009695,-0.00040312178643555,...,0.06474552556919999,0.0037219867270488,0.020390728582534,-0.004855609726982352,0.0035372942365255204,0.013201818435589,-0.0030404271371136,-0.0032332441818288,-7.10689383652776e-05,-0.0047330230107115595
3,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,Keep,1,9,1.0,健康医疗,-0.0454774110441126,0.12108299415182061,-0.0150667818984648,...,0.0106611927907857,0.0022276670574426,0.018511982163858604,-0.00528872129122992,0.0006997969256685561,-0.01073317343449638,-0.00205497725524466,-0.00242867009111637,0.00020931604237424758,-0.00390062219638372
4,B02AD69BFD4A4FEEDFAB6DBB36DCC167,7,开心消消乐,1,9,1.0,游戏服务,0.00423594427914244,-0.02816071338299894,0.027082874290590003,...,0.03266895085413,0.0034214489954958,0.057234973404225,0.0007363980335044479,-0.0033633517557612406,-0.015277080881689902,-0.0026509078813809,0.0015645956417787,-0.000214980965065152,-0.003144631266761308


In [328]:
data.to_csv('./data.csv', index=False)

In [330]:
len(data)

2436009

---
训练
---

---

In [336]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
# import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score

In [338]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data.iloc[:, 4:].values
y = data['click'].values
X.shape, y.shape

In [313]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [314]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 32,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

'[1, 2, 3]'

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=5000,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=50,
                    verbose_eval=50,
                    )

In [ ]:
f1_score(y_test, np.where(gbm.predict(X_test, num_iteration=gbm.best_iteration) > 0.5, 1, 0))